# Projet - Apprentissage Profond - Implémentation (Partie Shallow Network)

**EL KAAKOUR Ahmad & Matthieu RANDRIANTSOA**

> Ce notebook contient toutes les fonctions d'implémentation de nos modèles. Il est conçu de sorte à s'exécuter sequentiellement.

In [1]:
import gzip, numpy, torch
import matplotlib.pyplot as plt
from pathlib import Path
import itertools
import json
import os
import pandas as pd
import seaborn as sns

In [2]:
# Setup device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"

# Partie 2 : ShallowNetwork

In [6]:
batch_size = 5 # nombre de données lues à chaque fois
nb_epochs = 10 # nombre de fois que la base de données sera lue
eta = 0.00001 # taux d'apprentissage
w_min = -0.001
w_max = 0.001

## Chargement des données

In [ ]:
((data_train,label_train),(data_test,label_test)) = torch.load(gzip.open('data/mnist.pkl.gz'), weights_only=False)

## Préparation des données

In [ ]:
training_dataset = torch.utils.data.TensorDataset(data_train, label_train)
test_dataset = torch.utils.data.TensorDataset(data_test, label_test)

training_loader = torch.utils.data.DataLoader(training_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1, shuffle=True)

## Création du modèle de réseau de neurones

In [13]:
class ShallowNetwork(torch.nn.Module):
    def __init__(self, nb_of_neurons):
        super(ShallowNetwork, self).__init__()

        self.hidden_layer = torch.nn.Linear(28 * 28, nb_of_neurons)  # Entrées MNIST de taille 28x28
        self.output_layer = torch.nn.Linear(nb_of_neurons, 10)  # 10 classes MNIST (digits 0-9)

    def forward(self, x: torch.Tensor):
        x = self.hidden_layer(x)
        x = torch.nn.functional.relu(x)  # fonction d'activation pour introduire de la non-linearité
        x = self.output_layer(x)
        return x

## Entrainement du modèle

In [ ]:
shallowNetworkModel = ShallowNetwork(100)

In [ ]:
loss_func = torch.nn.CrossEntropyLoss() # Fonction coût utilisé pour la classification multi-classes
optim = torch.optim.SGD(shallowNetworkModel.parameters(), lr = 0.1)

In [ ]:
for n in range(nb_epochs):
    shallowNetworkModel.train() # Set model to training mode
    for x,t in training_loader:

        y = shallowNetworkModel(x)

        loss = loss_func(y, t) # Compare les prédictions y aux vraies valeurs t
        optim.zero_grad()
        loss.backward() # Calcul le gradient de la perte par rapport aux paramètres du modèles
        optim.step() #

    acc = 0
    shallowNetworkModel.eval()                      # Set model to evaluation mode
    with torch.inference_mode():                    # Disable gradient computation for testing
        for data, target in test_loader:
            outputs = shallowNetworkModel(data)

            loss = loss_func(outputs, target)
            acc += torch.argmax(outputs,1) == torch.argmax(target,1)

    print(f"Loss: {loss}, Accuracy: {(acc/data_test.shape[0]).numpy()}")

Loss: 0.000504723924677819, Accuracy: [0.93685716]
Loss: 0.12743769586086273, Accuracy: [0.9352857]
Loss: 0.000931544229388237, Accuracy: [0.939]
Loss: 0.6536549925804138, Accuracy: [0.93871427]
Loss: 0.055088337510824203, Accuracy: [0.94214284]
Loss: 3.492635726928711, Accuracy: [0.9392857]
Loss: 0.14079053699970245, Accuracy: [0.94]
Loss: 0.005089778918772936, Accuracy: [0.938]
Loss: 0.001042656716890633, Accuracy: [0.93814284]
Loss: 1.5437995195388794, Accuracy: [0.94285715]


## Enregistrer le modèle

In [ ]:
# 1. Create models directory
MODEL_PATH = Path("models")
MODEL_PATH.mkdir(parents=True, exist_ok=True)

# 2. Create model save path
MODEL_NAME = "shallow_network_130924CEL.pth"
MODEL_SAVE_PATH = MODEL_PATH / MODEL_NAME

# 3. Save the model state dict
print(f"Saving model to: {MODEL_SAVE_PATH}")
torch.save(obj=shallowNetworkModel.state_dict(), # only saving the state_dict() only saves the models learned parameters
           f=MODEL_SAVE_PATH)


Saving model to: models/shallow_network_130924CEL.pth


## Évaluation des performances

In [ ]:
# Evaluate the model on the test set
correct = 0

shallowNetworkModel.eval()                           # Set model to evaluation mode
with torch.inference_mode():                    # Disable gradient computation for testing
    for data, target in test_loader:
        outputs = shallowNetworkModel(data)
        correct += torch.argmax(outputs,1) == torch.argmax(target,1)

accuracy = 100 * correct/data_test.shape[0]
print(f'Test Accuracy: {accuracy.numpy().item()}%')

Test Accuracy: 94.28571319580078%


## Fonctions utilitaires

In [3]:
def train_and_evaluate(_model, _training_loader, _validation_loader, _params, device='cpu'):
    """
    Entraîne et évalue un modèle d'apprentissage automatique ou d'apprentissage profond.

    Paramètres:
      ----------
      _model : torch.nn.Module
          Le modèle à entraîner et à évaluer. Cela doit être un modèle PyTorch.

      _training_loader : torch.utils.data.DataLoader
          DataLoader pour le jeu de données d'entraînement, fournissant des lots de données à utiliser pour l'entraînement du modèle.

      _validation_loader : torch.utils.data.DataLoader
          DataLoader pour le jeu de données de validation, fournissant des lots de données pour évaluer le modèle après chaque époque.

      _params : dict
          Un dictionnaire contenant les hyperparamètres du modèle, comme le taux d'apprentissage, le nombre d'époques, et autres configurations spécifiques.

      device : str, optionnel
          L'appareil sur lequel exécuter le modèle (par exemple 'cpu' ou 'cuda' pour un GPU). Par défaut 'cpu'.
    """

    # Extraire les hyperparamètres depuis params
    _nb_epochs = _params.get('nb_epochs', 10)
    _loss_func = _params.get('loss_func', torch.nn.CrossEntropyLoss())
    _optim = _params.get('optimizer', torch.optim.SGD(_model.parameters(), lr=_params.get('learning_rate', 0.001)))

    # Déplacer le modèle vers le périphérique spécifié (GPU si disponible)
    _model.to(device)

    # Initialiser des listes pour suivre les pertes et les précisions afin d'analyser les performances
    training_losses = []
    validation_losses = []
    validation_accuracies = []

    # Boucle d'entraînement
    for epoch in range(_nb_epochs):
        _model.train()  # Mettre le modèle en mode entraînement
        epoch_training_loss = 0.0

        # Étape d'entraînement
        for x, t in _training_loader:
            x, t = x.to(device), t.to(device)  # Déplacer les données vers le périphérique

            # Passer en avant (forward pass)
            y = _model(x)

            # Calculer la perte
            _loss = _loss_func(y, t)
            epoch_training_loss += _loss.item()

            # Rétropropagation et optimisation
            _optim.zero_grad()  # Réinitialiser les gradients
            _loss.backward()  # Calculer les gradients
            _optim.step()  # Mettre à jour les poids du modèle

        # Perte moyenne d'entraînement pour l'époque
        avg_training_loss = epoch_training_loss / len(_training_loader)
        training_losses.append(avg_training_loss)

        # Étape de validation
        _model.eval()  # Mettre le modèle en mode évaluation
        epoch_val_loss = 0.0
        correct_predictions = 0
        total_predictions = 0

        # Désactiver le calcul des gradients pendant l'évaluation
        with torch.inference_mode():
            for data, target in _validation_loader:  # Utiliser validation_loader pour la validation
                data, target = data.to(device), target.to(device)  # Déplacer les données vers le périphérique

                # Passer en avant (forward pass)
                outputs = _model(data)

                # Calculer la perte
                _loss = _loss_func(outputs, target)
                epoch_val_loss += _loss.item()

                # Calculer la précision
                predicted_labels = torch.argmax(outputs, dim=1)
                true_labels = torch.argmax(target, dim=1)
                correct_predictions += (predicted_labels == true_labels).sum().item()
                total_predictions += target.size(0)

        # Perte moyenne de validation et précision pour l'époque
        avg_val_loss = epoch_val_loss / len(_validation_loader)
        validation_losses.append(avg_val_loss)

        accuracy = correct_predictions / total_predictions
        validation_accuracies.append(accuracy)

        # Afficher les métriques pour l'époque en cours
        print(f"Époque {epoch + 1}/{nb_epochs}, Validation loss: {avg_val_loss:.4f}, Validation accuracy: {accuracy:.4f}")

    # Calcul de la précision finale sur le jeu de test
    final_correct_predictions = 0
    final_total_predictions = 0

    _model.eval()  # Mettre le modèle en mode évaluation
    with torch.inference_mode():
        for data, target in _validation_loader:  # Utiliser test_loader pour la phase finale de test
            data, target = data.to(device), target.to(device)
            outputs = _model(data)
            predicted_labels = torch.argmax(outputs, dim=1)
            true_labels = torch.argmax(target, dim=1)
            final_correct_predictions += (predicted_labels == true_labels).sum().item()
            final_total_predictions += target.size(0)

    # Calculer la précision finale sur le jeu de test
    final_test_accuracy = final_correct_predictions / final_total_predictions
    print(f"Précision finale: {final_test_accuracy:.4f}")

    # Retourner les métriques de performance pour analyse ultérieure
    return {
        'hyperparameters': _params,
        'training_losses': training_losses,
        'validation_losses': validation_losses,
        'validation_accuracies': validation_accuracies,
        'final_validation_accuracy': final_test_accuracy
    }


In [4]:
def setup_and_train(_params, device='cpu'):
    """
    Prépare les données, les loader et le paramétrage du modèle. Puis lance l'entrainement et l'évaluation du modèle
    """
    _batch_size = _params.get('batch_size', 64)
    _nb_neurons = _params.get('nb_neurons', 10)

    # Create Loader
    train_loader = torch.utils.data.DataLoader(train_subset, batch_size=batch_size, shuffle=True)
    val_loader = torch.utils.data.DataLoader(val_subset, batch_size=batch_size, shuffle=False)

    # Model
    _model = ShallowNetwork(_nb_neurons)

    # Call training function
    print(f"Training the model : {_params.get('learning_rate', 0.001 ), _batch_size, nb_epochs, _nb_neurons}")
    return train_and_evaluate(_model, train_loader, val_loader, _params, device)

## Méthodologie pour trouver les bons hyperparamètres

### **Étape 1** : Préparation du jeu de validation

In [8]:
((data_train,label_train),(data_test,label_test)) = torch.load(gzip.open('data/mnist.pkl.gz'))
training_dataset = torch.utils.data.TensorDataset(data_train, label_train)
training_loader = torch.utils.data.DataLoader(training_dataset, batch_size=batch_size, shuffle=True)

In [9]:
# Diviser le jeu de données en sous-ensembles
generator = torch.Generator().manual_seed(42) # Permet de reproduire le même découpement
train_subset, val_subset = torch.utils.data.random_split(training_dataset, [0.8, 0.2], generator=generator)

In [10]:
print(len(train_subset), len(val_subset))

50400 12600


## **Étape 2** : Définir les hyperparamètres et définir une plage de valeurs à explorer

In [11]:
param_grid = {
    "learning_rate_range" :  [0.1, 0.01, 0.001, 0.0001],
    "batch_size_range" : [4, 8, 16, 32, 64, 128],
    "nb_epochs_range" :  [10, 20, 50, 100],
    "nb_neurones_range" :  [10, 25, 50, 100]
}

# Créer toutes les combinaisons de paramètres possibles
param_combinations = list(itertools.product(param_grid['learning_rate_range'],
                                            param_grid['batch_size_range'],
                                            param_grid['nb_epochs_range'],
                                            param_grid['nb_neurones_range'])
                          )

## **Étape 3** : Choisir une méthode d’optimisation des hyperparamètres (GridSearch)

In [ ]:
# Chemin vers le fichier de sortie
file_path = "data/model_metrics_1809024_follow_up.json"

# Vérifie si le fichier existe déjà pour savoir si c'est le premier ajout
first_write = not os.path.exists(file_path)

res = []

# Ouvre le fichier en mode ajout (append) pour écrire progressivement les résultats
with open(file_path, "a") as json_file:

    # Si c'est la première écriture, on ouvre un tableau JSON avec un crochet ouvrant
    if first_write:
        json_file.write("[\n")  # Début de l'array JSON

    # Boucle sur toutes les combinaisons d'hyperparamètres
    for i, (lr, batch_size, nb_epochs, hidden_units) in enumerate(param_combinations):

        # Calcule les résultats pour les hyperparamètres actuels
        result = setup_and_train(_params={
            "batch_size": batch_size,
            "nb_epochs": nb_epochs,
            "learning_rate": lr,
            "nb_neurons": hidden_units
        },
                      device=device)

        #
        res.append(result)

        # Écrit les résultats au format JSON dans le fichier
        json.dump(result, json_file)

        # Ajoute une virgule et un saut de ligne après chaque entrée, sauf la dernière
        if i < len(param_combinations) - 1:
            json_file.write(",\n")

    # Si c'est la première écriture, on ferme le tableau avec un crochet fermant après la boucle
    if first_write:
        json_file.write("\n]")


# Test final

In [14]:
# Params
_nb_neurons = 100
_nb_epochs = 50
_batch_size = 8
_learning_rate = 0.1

# Load data
((data_train,label_train),(data_test,label_test)) = torch.load(gzip.open('data/mnist.pkl.gz'), weights_only=False)
training_dataset = torch.utils.data.TensorDataset(data_train, label_train)
test_dataset = torch.utils.data.TensorDataset(data_test, label_test)
training_loader = torch.utils.data.DataLoader(training_dataset, batch_size=batch_size, shuffle=True)
generator = torch.Generator().manual_seed(42) # Permet de reproduire le même découpement
train_subset, val_subset = torch.utils.data.random_split(training_dataset, [0.8, 0.2], generator=generator)
train_loader = torch.utils.data.DataLoader(train_subset, batch_size=8, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_subset, batch_size=8, shuffle=False)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1, shuffle=False)


model = ShallowNetwork(_nb_neurons)

#
_loss_func = torch.nn.CrossEntropyLoss()
_optimizer = torch.optim.SGD(model.parameters(), lr=_learning_rate)


# Move model to device
model.to(device)

# Training loop
for epoch in range(_nb_epochs):
    model.train()  # Set model to training mode
    for x, t in train_loader:
        x, t = x.to(device), t.to(device)  # Move data to device

        # Forward pass
        y = model(x)

        # Compute loss
        loss = _loss_func(y, t)

        # Backpropagation and optimization
        _optimizer.zero_grad()  # Reset gradients
        loss.backward()  # Compute gradients
        _optimizer.step()  # Update model weights

    # Validation step (optional)
    model.eval()  # Set model to evaluation mode
    with torch.inference_mode():
        for data, target in val_loader:
            data, target = data.to(device), target.to(device)
            outputs = model(data)
            loss = _loss_func(outputs, target)

# save model
MODEL_PATH = Path("models")
MODEL_PATH.mkdir(parents=True, exist_ok=True)

# 2. Create model save path
MODEL_NAME = "shallow_network_130924CEL.pth"
MODEL_SAVE_PATH = MODEL_PATH / MODEL_NAME

# 3. Save the model state dict
print(f"Saving model to: {MODEL_SAVE_PATH}")
torch.save(obj=model.state_dict(), # only saving the state_dict() only saves the models learned parameters
           f=MODEL_SAVE_PATH)

acc = 0.
# on lit toutes les donnéees de test
for x,t in test_loader:
  # on calcule la sortie du modèle
  y = model(x)
  # on regarde si la sortie est correcte
  acc += torch.argmax(y,1) == torch.argmax(t,1)
# on affiche le pourcentage de bonnes réponses
print(acc/data_test.shape[0])

Saving model to: models\shallow_network_130924CEL.pth
tensor([0.9831])


In [27]:
# Load model
model = ShallowNetwork(_nb_neurons)
model.load_state_dict(torch.load("models/model_shallow_network.pth"))

<All keys matched successfully>

In [28]:
# Evaluate the model on the test set
correct = 0

model.eval()                           # Set model to evaluation mode
with torch.inference_mode():                    # Disable gradient computation for testing
    for data, target in test_loader:
        outputs = model(data)
        correct += torch.argmax(outputs,1) == torch.argmax(target,1)
        
accuracy = 100 * correct/data_test.shape[0]
print(f'Test Accuracy: {accuracy.numpy().item()}%')

Test Accuracy: 98.31428527832031%
